# On device Training Little

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

## Load Models

In [2]:
model = tf.keras.models.load_model("models/emnist_model.keras")

In [4]:
inf = tf.lite.Interpreter(model_path="models/emnist_litert_infer.tflite")
inf.allocate_tensors()
inp_det = inf.get_input_details()[0]
out_det = inf.get_output_details()[0]

## Load test dataset

In [6]:
element_spec = (
    tf.TensorSpec([None, 28, 28, 1], tf.float32),
    tf.TensorSpec([None],          tf.int64),
)
ds_test_proc = tf.data.Dataset.load("saved_ds/test_proc", element_spec)

### Prepare DS and Model

In [7]:
x_list, y_list = [], []
for xb, yb in ds_test_proc.take(2):
    x_list.append(xb.numpy())
    y_list.append(yb.numpy())
    
x = np.concatenate(x_list, axis=0)   # shape (128,28,28,1)
y = np.concatenate(y_list, axis=0)   # shape (128,)

## Compare

In [9]:
num = 100
keras_logits  = model.predict(x[:num])
tflite_logits = []
for i in range(num):
    sample = x[i:i+1].astype(np.float32)
    inf.set_tensor(inp_det['index'], sample)
    inf.invoke()
    tflite_logits.append(inf.get_tensor(out_det['index'])[0])
tflite_logits = np.stack(tflite_logits, axis=0)

keras_preds  = np.argmax(keras_logits, axis=1)
tflite_preds = np.argmax(tflite_logits,axis=1)
kacc         = np.mean(keras_preds  == y[:num])
tacc         = np.mean(tflite_preds == y[:num])
avgd         = np.mean(np.abs(keras_logits - tflite_logits))
maxd         = np.max (np.abs(keras_logits - tflite_logits))

print("📊 Model Comparison Summary")
print(f"✅ Keras Accuracy:      {kacc:.4f}")
print(f"✅ TFLite Accuracy:     {tacc:.4f}")
print(f"📉 Avg Logit Delta:     {avgd:.2e}")
print(f"📈 Max Logit Delta:     {maxd:.2e}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step
📊 Model Comparison Summary
✅ Keras Accuracy:      0.9500
✅ TFLite Accuracy:     0.9500
📉 Avg Logit Delta:     1.26e-08
📈 Max Logit Delta:     1.58e-06
